In [69]:
import requests

# 요청 URL
url = "https://mapi.ticketlink.co.kr/mapi/sports/reserveOpen?categoryId=137"

# 요청 헤더 (개발자 도구에서 가져온 값 활용)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.ticketlink.co.kr/",
    "Origin": "https://www.ticketlink.co.kr",
    "x-access-token": "AAAAmAxeyruGusS7iEoBb7M-6zTttaocxlKzL0E9fX5C8kYdAL24Ea63hkUE1t8DsX3w5OXRkRY3mZWZElH-LLxhJacTtJuehXGD5m8XloOT1ihw-k6fNLQ-Gy50xZcDjFzJJTGonlRlmLaBGUKw-QjmAb4myQe3t6KTTeGMP--3cLHcFl64_F79_q7cq-SUjNw8mCZ1pu_lRpJYHg_QhKGDeFY.A",
    "x-tl-locale": "ko",
}

# API 요청
response = requests.get(url, headers=headers)

# 응답 확인
if response.status_code == 200:
    data = response.json()  # JSON 응답 처리
    # print(data)  # 출력
else:
    print(f"요청 실패: {response.status_code}")

In [70]:
from datetime import datetime

def str_from_timestamp(timestamp):
    date = datetime.fromtimestamp(timestamp / 1000)
    date = date.strftime("%Y-%m-%d %H:%M:%S")
    return date

In [71]:
result = list()

for game in data["data"]:
    scheduleId = str(game["scheduleId"])
    productId = str(game["productId"])
    title = game["matchTitle"]
    homeTeam = game["homeTeam"]["teamName"]
    awayTeam = game["awayTeam"]["teamName"]
    field = game["venueName"]
    schedule = str_from_timestamp(game["scheduleDate"])
    openTime = str_from_timestamp(game["reserveOpenDateTime"])
    link = f"https://www.ticketlink.co.kr/reserve/product/{productId}?scheduleId={scheduleId}"
    result.append({"date": schedule, "home_team": homeTeam, "away_team": awayTeam, "field": field, "open_time": openTime, "link": link, "game_title": title, "schedule_id": scheduleId, "product_id": productId})
    
result = sorted(result, key=lambda x:x["date"])

In [72]:
import pandas as pd
# pd.DataFrame(result)
pd.DataFrame(result).to_excel("test.xlsx", index=False)

In [74]:
import requests
import json
import time
from datetime import datetime

def fetch_ticketlink_api():
    # 대상 API URL
    api_url = "https://mapi.ticketlink.co.kr/mapi/sports/reserveOpen"
    
    # 요청 파라미터
    params = {
        "categoryId": "139"
    }
    
    # 요청 헤더 (실제 브라우저처럼 보이도록 설정)
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Referer": "https://www.ticketlink.co.kr/",
        "Origin": "https://www.ticketlink.co.kr",
        "Connection": "keep-alive"
    }
    
    try:
        # API 요청
        response = requests.get(api_url, params=params, headers=headers)
        
        # 상태 코드 확인
        if response.status_code == 200:
            # JSON 응답 확인
            data = response.json()
            
            # 현재 시간으로 파일명 생성
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"ticketlink_sports_data_{timestamp}.json"
            
            # JSON 파일로 저장
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
                
            print(f"API 응답을 '{filename}' 파일로 저장했습니다.")
            
            # 응답 데이터 출력 (간략히)
            print("\n응답 데이터 미리보기:")
            if isinstance(data, dict):
                for key in data:
                    print(f"{key}: {type(data[key])}")
            elif isinstance(data, list):
                print(f"총 {len(data)}개의 항목이 있습니다.")
                if data and len(data) > 0:
                    print("첫 번째 항목 미리보기:")
                    for key in data[0]:
                        print(f"  {key}: {data[0][key]}")
            
            return data
        else:
            print(f"API 요청 실패: 상태 코드 {response.status_code}")
            print("응답 내용:", response.text)
            return None
            
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return None

# 주기적으로 API 호출 (선택 사항)
def monitor_api(interval_minutes=30, duration_hours=24):
    """
    지정된 시간 동안 주기적으로 API를 모니터링합니다.
    
    Args:
        interval_minutes: 호출 간격 (분)
        duration_hours: 총 모니터링 시간 (시간)
    """
    total_calls = int((duration_hours * 60) / interval_minutes)
    
    print(f"{duration_hours}시간 동안 {interval_minutes}분 간격으로 API를 호출합니다. (총 {total_calls}회)")
    
    for i in range(total_calls):
        print(f"\n[{i+1}/{total_calls}] {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} API 호출 중...")
        fetch_ticketlink_api()
        
        if i < total_calls - 1:  # 마지막 호출이 아니면 대기
            print(f"{interval_minutes}분 대기 중...")
            time.sleep(interval_minutes * 60)

if __name__ == "__main__":
    # API 한 번 호출
    print("티켓링크 API 호출 중...")
    fetch_ticketlink_api()

티켓링크 API 호출 중...
API 응답을 'ticketlink_sports_data_20250319_002509.json' 파일로 저장했습니다.

응답 데이터 미리보기:
data: <class 'list'>
result: <class 'dict'>
success: <class 'bool'>
